In [115]:
import pandas as pd
patient = pd.read_csv('PatientRoute.csv')
patient = patient[(patient['province']=='Seoul')]
patient = patient.drop_duplicates()

In [116]:
patient['date'] = pd.to_datetime(patient['date'])

In [117]:
def last_loc(patient):
    route = []
    for pat in patient['patient_id'].unique():
        uni_patient = patient[patient['patient_id'] == pat]
        uni_patient = uni_patient.sort_values(by=['date'])
        uni_patient = uni_patient.iloc[-1,:]
        uni_patient = pd.DataFrame(uni_patient).T
        route.append(uni_patient)
    return pd.concat(route)

In [118]:
def minus_num(num):
    route = []
    for pat in patient['patient_id'].unique():
        uni_patient = patient[patient['patient_id'] == pat]
        uni_patient = uni_patient.sort_values(by=['date'])
        each_day = uni_patient.iloc[-1,:]['date']
        poi = uni_patient[(uni_patient['date']<=each_day)&(each_day-pd.Timedelta(days=num)<=uni_patient['date'])]
        route.append(poi)
    return pd.concat(route)

In [119]:
last_location = last_loc(patient)
minum_0 = minus_num(0)
minum_1 = minus_num(1)
minum_2 = minus_num(2)
minum_3 = minus_num(3)

In [120]:
from scipy.spatial.distance import pdist, squareform
import geopy.distance
def dist_metric(coord1, coord2):
    coords_1 = (coord1[0], coord1[1])
    coords_2 = (coord2[0], coord2[1])
    return geopy.distance.vincenty(coords_1, coords_2).km

In [121]:
def dist_dataframe(p):
    pdis = []
    for i, dat in enumerate(p['date'].unique()):
        route1 = p[p['date'] == dat]
        tmp = pdist(route1[['latitude', 'longitude']].values, dist_metric)
        if len(tmp)>0:
            maximum = np.max(tmp)
            avg = np.mean(tmp)
            minimum = np.min(tmp)
            stdev = np.std(tmp)
            length = len(route1['patient_id'].unique())
            pdis.append([pd.to_datetime(dat).strftime('%Y-%m-%d'), avg, minimum, maximum, stdev, length])
    pair = pd.DataFrame(pdis)
    pair.columns = ['date','average','minimum','maximum','stdev','length']
    pair = pair.merge(date_match, on='date', how='right').sort_values(by=['date'], axis=0).reset_index().replace(np.nan, 0)
    pair = pair[['date','average','minimum','maximum','stdev','length','num']]
    return pair

In [122]:
date_match = pd.DataFrame(range(1, 103), pd.date_range(start="2020-01-20",end="2020-04-30")).reset_index()
date_match.columns = ['date','num']
date_match['date'] = date_match['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [123]:
import numpy as np
last_seoul = dist_dataframe(last_location)
minus_0 = dist_dataframe(minum_0)
minus_1 = dist_dataframe(minum_1)
minus_2 = dist_dataframe(minum_2)
minus_3 = dist_dataframe(minum_3)

C:\Users\Main\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [124]:
patientinfo = pd.read_csv('PatientInfo.csv')
region = pd.read_csv('region.csv')
patientinfo = patientinfo[patientinfo['country']=='Korea']
patientinfo = patientinfo[['patient_id','province','city','confirmed_date']]
patientinfo = patientinfo[~((patientinfo['city'].isnull())|(patientinfo['confirmed_date'].isnull()))]
region = region[['province','city','latitude','longitude']]
patientinfo = patientinfo.merge(region, how='left', left_on=['province','city'], right_on=['province','city'])
patientinfo[patientinfo['latitude'].isnull()] = patientinfo[patientinfo['latitude'].isnull()].merge(region, how='left',left_on='province',right_on='province')
patientinfo = patientinfo[~patientinfo['latitude'].isnull()]
patientinfo.columns = ['patient_id','province','city','date','latitude','longitude']
patientinfo['date'] = pd.to_datetime(patientinfo['date'])
last_location1 = last_loc(patientinfo)
last_nation = dist_dataframe(last_location1)

C:\Users\Main\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [127]:
last_seoul[45:55]

,date,average,minimum,maximum,stdev,length,num
45,2020-03-05,14.435757,0.000000,22.125157,8.157169,5.0,46
46,2020-03-06,8.924990,0.000000,22.532003,5.780228,13.0,47
47,2020-03-07,11.459303,0.000000,25.944409,5.931445,14.0,48
48,2020-03-08,14.375115,3.208630,25.944409,6.378359,7.0,49
49,2020-03-09,7.462318,0.000000,27.487702,7.388526,28.0,50
50,2020-03-10,12.463523,0.000000,25.944409,9.028304,29.0,51
51,2020-03-11,13.063158,0.000000,25.944409,10.009189,21.0,52
52,2020-03-12,12.781139,0.000000,25.944409,7.263814,13.0,53
53,2020-03-13,12.441334,0.000000,23.282193,7.563056,9.0,54
54,2020-03-14,11.763293,5.240509,20.303545,4.882168,5.0,55


In [128]:
last_nation[45:55]

,date,average,minimum,maximum,stdev,length,num
45,2020-03-05,95.020141,0.0,329.638097,72.964730,143.0,46
46,2020-03-06,101.521444,0.0,336.641755,90.792422,97.0,47
47,2020-03-07,129.954225,0.0,329.091566,93.481449,55.0,48
48,2020-03-08,134.874604,0.0,288.578705,94.573457,51.0,49
49,2020-03-09,101.951821,0.0,293.849392,104.467472,65.0,50
50,2020-03-10,108.433521,0.0,359.728540,101.265248,95.0,51
51,2020-03-11,127.045889,0.0,347.648548,89.855040,50.0,52
52,2020-03-12,96.774029,0.0,316.232564,85.695531,34.0,53
53,2020-03-13,112.788138,0.0,337.255461,112.373271,34.0,54
54,2020-03-14,129.550863,0.0,343.491910,109.960936,33.0,55


In [129]:
last_seoul.to_csv('seoul_lastlocation.csv')
last_nation.to_csv('nation_lastlocation.csv')